# Grouping

1. Simple grouping
2. More complex grouping
    - Grouping on more than one categorical column
    - Grouping on more than one numeric column
    - Using more than one aggregation method
3. Pivot tables (2D grouping -- on two categorical columns)
4. Stacking and unstacking -- moving things from rows to columns and back

In [2]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame

In [3]:
filename = '../data/taxi.csv'   # 10k taxi rides from NYC in 2015
df = pd.read_csv(filename)

In [4]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,2,2015-06-02 11:19:29,2015-06-02 11:47:52,1,1.63,-73.954430,40.764141,1,N,-73.974754,40.754093,2,17.0,0.0,0.5,0.00,0.0,0.3,17.80
1,2,2015-06-02 11:19:30,2015-06-02 11:27:56,1,0.46,-73.971443,40.758942,1,N,-73.978539,40.761909,1,6.5,0.0,0.5,1.00,0.0,0.3,8.30
2,2,2015-06-02 11:19:31,2015-06-02 11:30:30,1,0.87,-73.978111,40.738434,1,N,-73.990273,40.745438,1,8.0,0.0,0.5,2.20,0.0,0.3,11.00
3,2,2015-06-02 11:19:31,2015-06-02 11:39:02,1,2.13,-73.945892,40.773529,1,N,-73.971527,40.760330,1,13.5,0.0,0.5,2.86,0.0,0.3,17.16
4,1,2015-06-02 11:19:32,2015-06-02 11:32:49,1,1.40,-73.979088,40.776772,1,N,-73.982162,40.758999,2,9.5,0.0,0.5,0.00,0.0,0.3,10.30


In [6]:
# I want the mean trip_distance where passenger_count is 1

df.loc[
    df['passenger_count'] == 1     # row selector
    ,
    'trip_distance'    # column selector
].mean()

3.0923380047176354

In [7]:
# but what if I want to perform the same calculation for passenger_count == 2

df.loc[
    df['passenger_count'] == 2     # row selector
    ,
    'trip_distance'    # column selector
].mean()

3.3843869002284848

In [8]:
# but what if I want to perform the same calculation for passenger_count == 3

df.loc[
    df['passenger_count'] == 3     # row selector
    ,
    'trip_distance'    # column selector
].mean()

3.3423891625615765

What we're doing (manually) is taking each distinct/unique value in `passenger_count` and we're running our query on it.

The whole point of grouping is to ask Pandas to do this same task for us, calculating once for each distinct value in `passenger_count`.

The way to think about grouping is as follows:

- One categorical column, on which we'll do the grouping. We'll get one result for each distinct value in this column.
- One numeric column, on which we'll perform the calculation.
- One aggregation method, which will be invoked on all of the values in the numeric column for each distinct value of the categorical

In our example above:
- passenger_count is categorical
- trip_distance is numeric
- mean is an aggregation method

In [9]:
# df.groupby(CATEGORICAL)[NUMERIC].aggregate()

df.groupby('passenger_count')['trip_distance'].mean()

passenger_count
0    4.600000
1    3.092338
2    3.384387
3    3.342389
4    3.628901
5    3.182712
6    3.170976
Name: trip_distance, dtype: float64

In [10]:
# normally, it's not a bad thing that Pandas automatically sorts the distinct values in 
# passenger_count and then displays them in that order. However, if you're going to reorder
# them, or if you want to save a bit of calculation, you can pass the sort=False keyword
# argument.

df.groupby('passenger_count', sort=False)['trip_distance'].mean()

passenger_count
1    3.092338
4    3.628901
3    3.342389
2    3.384387
5    3.182712
6    3.170976
0    4.600000
Name: trip_distance, dtype: float64

In [11]:
# what does groupby return?

df.groupby('passenger_count')['trip_distance']

In [12]:
df.groupby('passenger_count', sort=False)['trip_distance'].mean().sort_index(ascending=False)

passenger_count
6    3.170976
5    3.182712
4    3.628901
3    3.342389
2    3.384387
1    3.092338
0    4.600000
Name: trip_distance, dtype: float64

In [13]:
filename = '../data/olympic_athlete_events.csv'   # all athletes from all Olympic games through 2020

!head $filename

"ID","Name","Sex","Age","Height","Weight","Team","NOC","Games","Year","Season","City","Sport","Event","Medal"
"1","A Dijiang","M",24,180,80,"China","CHN","1992 Summer",1992,"Summer","Barcelona","Basketball","Basketball Men's Basketball",NA
"2","A Lamusi","M",23,170,60,"China","CHN","2012 Summer",2012,"Summer","London","Judo","Judo Men's Extra-Lightweight",NA
"3","Gunnar Nielsen Aaby","M",24,NA,NA,"Denmark","DEN","1920 Summer",1920,"Summer","Antwerpen","Football","Football Men's Football",NA
"4","Edgar Lindenau Aabye","M",34,NA,NA,"Denmark/Sweden","DEN","1900 Summer",1900,"Summer","Paris","Tug-Of-War","Tug-Of-War Men's Tug-Of-War","Gold"
"5","Christine Jacoba Aaftink","F",21,185,82,"Netherlands","NED","1988 Winter",1988,"Winter","Calgary","Speed Skating","Speed Skating Women's 500 metres",NA
"5","Christine Jacoba Aaftink","F",21,185,82,"Netherlands","NED","1988 Winter",1988,"Winter","Calgary","Speed Skating","Speed Skating Women's 1,000 metres",NA
"5","Christine Jacoba Aaftink","F",25,1

# Exercise: Olympic calculations

1. Read the Olympic data into a data frame.
2. Find the mean age for people in each sport.
3. Find the max height for people in each sport after 1960.

In [14]:
df = pd.read_csv(filename)

In [15]:
df.shape

(271116, 15)

In [16]:
# find the mean age for people in each sport

df.groupby('Sport')['Age'].mean()

Sport
Aeronautics         26.000000
Alpine Skiing       23.205462
Alpinism            38.812500
Archery             27.935226
Art Competitions    45.901009
                      ...    
Tug-Of-War          29.309524
Volleyball          25.183800
Water Polo          25.659627
Weightlifting       25.502010
Wrestling           25.798289
Name: Age, Length: 66, dtype: float64

In [17]:
df.groupby('Sport')['Age'].mean().sort_values()

Sport
Rhythmic Gymnastics      18.737082
Swimming                 20.566803
Figure Skating           22.232190
Synchronized Swimming    22.366851
Diving                   22.481441
                           ...    
Equestrianism            34.390831
Polo                     35.333333
Alpinism                 38.812500
Art Competitions         45.901009
Roque                    53.333333
Name: Age, Length: 66, dtype: float64

In [22]:
# Find the max height for people in each sport after 1960.

(
    df
    .loc[df['Year'] > 1960]
    .groupby('Sport')['Height'].max()
)

Sport
Alpine Skiing                200.0
Archery                      197.0
Athletics                    208.0
Badminton                    201.0
Baseball                     206.0
Basketball                   226.0
Beach Volleyball             212.0
Biathlon                     200.0
Bobsleigh                    205.0
Boxing                       205.0
Canoeing                     205.0
Cross Country Skiing         200.0
Curling                      197.0
Cycling                      201.0
Diving                       188.0
Equestrianism                197.0
Fencing                      208.0
Figure Skating               193.0
Football                     200.0
Freestyle Skiing             197.0
Golf                         196.0
Gymnastics                   185.0
Handball                     214.0
Hockey                       200.0
Ice Hockey                   206.0
Judo                         213.0
Luge                         199.0
Modern Pentathlon            198.0
Nordic Combine

In [23]:
# can I groupby on a numeric column?
# yes - but be careful!

df = pd.read_csv('../data/taxi.csv')

# for every distinct value of trip_distance...
# get the mean total_amount
df.groupby('trip_distance')['total_amount'].mean()  

trip_distance
0.00      31.58194
0.01      52.80000
0.02      43.46000
0.03       3.96000
0.04      70.01000
           ...    
34.84    137.59000
35.51    135.13000
37.20    210.14000
60.30    160.05000
64.60     79.96000
Name: total_amount, Length: 1219, dtype: float64

# More complex grouping

What if we want (with the taxi data) to group not just by passenger_count, but also by vendorID (i.e., who made the taxi meter)?

Meaning: I want to know the mean `trip_distance` not just for every distinct value of `passenger_count`, but for every combination of `passenger_count` and `VendorID`. 

This requires that we remember a general rule of thumb in Pandas: Wherever you can pass/use a single column name, you can also pass a list of column names.

In [24]:
df.groupby(['passenger_count', 'VendorID'])['trip_distance'].mean()

passenger_count  VendorID
0                1           4.600000
1                1           2.956456
                 2           3.262967
2                1           3.452027
                 2           3.328849
3                1           3.588535
                 2           3.187189
4                1           3.952239
                 2           3.440522
5                1           4.933333
                 2           3.172553
6                2           3.170976
Name: trip_distance, dtype: float64

# From a two-column groupby, we get a "multi-index"

This mean: A two-dimensional index. If I retrieve row 4, then I'll get all of the rows for it

In [26]:
df.groupby(['passenger_count', 'VendorID'])['trip_distance'].mean().loc[4]

VendorID
1    3.952239
2    3.440522
Name: trip_distance, dtype: float64

In [27]:
# normally, we get the subset of the data frame for our named row
# but we can use multiple row indexes (fancy indexing) to get more than one
df.groupby(['passenger_count', 'VendorID'])['trip_distance'].mean().loc[[4, 6]]

passenger_count  VendorID
4                1           3.952239
                 2           3.440522
6                2           3.170976
Name: trip_distance, dtype: float64

# What about naming more than one numeric column?

Let's ask for the total_amount and trip_distance for each distinct value of passenger_count

In [29]:
df.groupby('passenger_count')[['trip_distance', 'total_amount']].mean()

,trip_distance,total_amount
passenger_count,,
0,4.600000,25.570000
1,3.092338,17.368569
2,3.384387,18.406306
3,3.342389,17.994704
4,3.628901,18.881648
5,3.182712,17.211269
6,3.170976,17.401355


In [30]:
df.groupby('passenger_count')[['trip_distance', 'total_amount']].mean()['trip_distance']

passenger_count
0    4.600000
1    3.092338
2    3.384387
3    3.342389
4    3.628901
5    3.182712
6    3.170976
Name: trip_distance, dtype: float64

# What about more than one aggregation method?

Just as we can pass a list of strings for columns (for the categorical or the numeric), we can do just about the same thing with the method. The difference is that we pass a list of strings to the `agg` method.

In [31]:
df.groupby('passenger_count')['trip_distance'].agg(['mean', 'min'])

,mean,min
passenger_count,,
0,4.600000,1.3
1,3.092338,0.0
2,3.384387,0.0
3,3.342389,0.0
4,3.628901,0.0
5,3.182712,0.0
6,3.170976,0.0


In [32]:
# what if I want to combine these?
# can I have two numeric columns and also two aggregation methods?

# OF COURSE!

# we're asking for:
# for each distinct value in passenger_count
# calculate the mean and min trip_distance and total_amount

df.groupby('passenger_count')[['trip_distance', 'total_amount']].agg(['mean', 'min'])

trip_distance      total_amount       
                         mean  min         mean    min
passenger_count                                       
0                    4.600000  1.3    25.570000  14.75
1                    3.092338  0.0    17.368569  -7.80
2                    3.384387  0.0    18.406306   0.30
3                    3.342389  0.0    17.994704   4.80
4                    3.628901  0.0    18.881648   3.30
5                    3.182712  0.0    17.211269   3.30
6                    3.170976  0.0    17.401355   3.30

In [34]:
# don't do this!

# the normal syntax is
# df.groupby(categorical_column)[numeric_column].agg([list_of_agg_methods])
# - more than one categorical column, we need a list, and thus strings in square brackets
# - the numeric column is always in square brackets, so if we have two, we need the list
df.groupby(['passenger_count', 'VendorID'])[['trip_distance', 'total_amount']].agg(['mean', 'min'])

trip_distance      total_amount       
                                  mean  min         mean    min
passenger_count VendorID                                       
0               1             4.600000  1.3    25.570000  14.75
1               1             2.956456  0.0    16.941386   0.30
                2             3.262967  0.0    17.904989  -7.80
2               1             3.452027  0.0    19.076807   0.30
                2             3.328849  0.0    17.855770   2.30
3               1             3.588535  0.0    19.002803   4.80
                2             3.187189  0.0    17.359076   4.80
4               1             3.952239  0.0    20.518657   5.80
                2             3.440522  0.0    17.927913   3.30
5               1             4.933333  3.0    20.466667  13.80
                2             3.172553  0.0    17.192379   3.30
6               2             3.170976  0.0    17.401355   3.30

In [36]:
def sillyfunc(s):
    return s * 2

In [37]:
df.groupby('passenger_count')['trip_distance'].agg(['sillyfunc'])

AttributeError: 'SeriesGroupBy' object has no attribute 'sillyfunc'

# More Olympic grouping!

1. Find, for each combination of year and team, the tallest athlete. (Another way to describe it: Find the tallest athlete across each year in the team.)
2. Find, for each year, the youngest and oldest athletes for each team.
3. Find, for each sport, the mean and median weights.
4. Find, for each combination of year/country, the min and max age and height.

In [38]:
df = pd.read_csv('../data/olympic_athlete_events.csv')

In [41]:
# Find, for each combination of year and team, the tallest athlete. 
# (Another way to describe it: Find the tallest athlete across each year in the team.)

(
    df
    .groupby(['Year', 'Team'])['Height'].max()
    .sort_values(ascending=False)
    .head(5)
)

Year  Team     
2004  China        226.0
2008  China        226.0
2000  China        226.0
1992  Lithuania    223.0
1996  Lithuania    223.0
Name: Height, dtype: float64

In [43]:
# Find, for each year, the youngest and oldest athletes for each team.

(
    df
    .groupby('Year')['Age'].agg(['min', 'max'])
)

,min,max
Year,,
1896,10.0,40.0
1900,13.0,71.0
1904,14.0,71.0
1906,13.0,54.0
1908,14.0,61.0
1912,13.0,67.0
1920,13.0,72.0
1924,11.0,81.0
1928,11.0,97.0


In [44]:
# find the values for all years from 1980 - 1992

(
    df
    .groupby('Year')['Age'].agg(['min', 'max'])
    .loc[1980:1992]
)

,min,max
Year,,
1980,13.0,70.0
1984,12.0,60.0
1988,11.0,70.0
1992,11.0,62.0


In [45]:
# Find, for each sport, the mean and median weights.

(
    df
    .groupby('Sport')['Weight'].agg(['mean', 'median'])
)

,mean,median
Sport,,
Aeronautics,NaN,NaN
Alpine Skiing,72.068110,71.0
Alpinism,NaN,NaN
Archery,70.011135,69.0
Art Competitions,75.290909,76.0
...,...,...
Tug-Of-War,95.615385,95.0
Volleyball,78.900214,78.0
Water Polo,84.566446,84.0


In [47]:
# Find, for each combination of year/country, the min and max age and height.

results = (
    df
    .groupby(['Year', 'Team'])[['Age', 'Height']].agg(['min', 'max'])
)
results

Age       Height       
                                     min   max    min    max
Year Team                                                   
1896 Australia                      22.0  22.0    NaN    NaN
     Australia/Great Britain        22.0  23.0    NaN    NaN
     Austria                        19.0  23.0    NaN    NaN
     Denmark                        21.0  34.0    NaN    NaN
     Ethnikos Gymnastikos Syllogos  10.0  10.0    NaN    NaN
...                                  ...   ...    ...    ...
2016 Venezuela                      19.0  52.0  153.0  206.0
     Vietnam                        19.0  42.0  155.0  185.0
     Yemen                          16.0  24.0  166.0  174.0
     Zambia                         20.0  28.0  170.0  182.0
     Zimbabwe                       21.0  42.0  155.0  191.0

[5170 rows x 4 columns]

In [49]:
results.loc[[2000, 2016]]

Age       Height       
                      min   max    min    max
Year Team                                    
2000 Albania         21.0  40.0  159.0  182.0
     Algeria         18.0  34.0  146.0  190.0
     American Samoa  16.0  34.0  168.0  188.0
     Andorra         20.0  49.0  162.0  188.0
     Angola          17.0  35.0  159.0  208.0
...                   ...   ...    ...    ...
2016 Venezuela       19.0  52.0  153.0  206.0
     Vietnam         19.0  42.0  155.0  185.0
     Yemen           16.0  24.0  166.0  174.0
     Zambia          20.0  28.0  170.0  182.0
     Zimbabwe        21.0  42.0  155.0  191.0

[492 rows x 4 columns]

In [50]:
results['Age']

min   max
Year Team                                     
1896 Australia                      22.0  22.0
     Australia/Great Britain        22.0  23.0
     Austria                        19.0  23.0
     Denmark                        21.0  34.0
     Ethnikos Gymnastikos Syllogos  10.0  10.0
...                                  ...   ...
2016 Venezuela                      19.0  52.0
     Vietnam                        19.0  42.0
     Yemen                          16.0  24.0
     Zambia                         20.0  28.0
     Zimbabwe                       21.0  42.0

[5170 rows x 2 columns]

In [51]:
# what if I just want min + age ?
# There, we use a tuple!

results[('Age', 'min')]

Year  Team                         
1896  Australia                        22.0
      Australia/Great Britain          22.0
      Austria                          19.0
      Denmark                          21.0
      Ethnikos Gymnastikos Syllogos    10.0
                                       ... 
2016  Venezuela                        19.0
      Vietnam                          19.0
      Yemen                            16.0
      Zambia                           20.0
      Zimbabwe                         21.0
Name: (Age, min), Length: 5170, dtype: float64

In [52]:
# what if I want, from the index (on the rows), just those rows where the country is 'United States'?
# my favorite way is with .xs

results.xs('United States',    # find this value
           level='Team')       # in this part of the multi-index

Age       Height       
       min   max    min    max
Year                          
1896  18.0  40.0  170.0  188.0
1900  17.0  44.0  160.0  188.0
1904  15.0  71.0  160.0  195.0
1906  18.0  37.0  167.0  195.0
1908  16.0  56.0  160.0  195.0
1912  17.0  60.0  157.0  195.0
1920  13.0  48.0  142.0  192.0
1924  13.0  50.0  142.0  198.0
1928  12.0  97.0  147.0  198.0
1932  13.0  96.0  147.0  198.0
1936  13.0  74.0  158.0  205.0
1948  15.0  51.0  140.0  213.0
1952  15.0  58.0  157.0  213.0
1956  14.0  49.0  152.0  207.0
1960  13.0  46.0  150.0  211.0
1964  14.0  46.0  150.0  213.0
1968  14.0  54.0  149.0  206.0
1972  14.0  49.0  149.0  223.0
1976  14.0  47.0  147.0  208.0
1980  15.0  32.0  153.0  193.0
1984  15.0  49.0  142.0  213.0
1988  14.0  52.0  142.0  216.0
1992  14.0  54.0  139.0  216.0
1994  16.0  36.0  155.0  193.0
1996  14.0  50.0  139.0  216.0
1998  15.0  40.0  147.0  195.0
2000  15.0  46.0  150.0  207.0
2002  16.0  48.0  155.0  193.0
2004  15.0  52.0  147.0  213.0
2006  16.0  44.0  152.0  195.0
2008  15.0  58.0  145.0  211.0
2010  16.0  40.0  152.0  193.0
2012  15.0  54.0  150.0  216.0
2014  15.0  45.0  152.0  196.0
2016  16.0  52.0  143.0  211.0

In [53]:
# this gives me back a series
df.groupby('Sport')['Weight'].mean()

Sport
Aeronautics               NaN
Alpine Skiing       72.068110
Alpinism                  NaN
Archery             70.011135
Art Competitions    75.290909
                      ...    
Tug-Of-War          95.615385
Volleyball          78.900214
Water Polo          84.566446
Weightlifting       78.726663
Wrestling           75.495570
Name: Weight, Length: 66, dtype: float64

In [54]:
# this gives me back a data frame, because we asked for a list of numeric columns, not one numeric column
df.groupby('Sport')[['Weight']].mean()

,Weight
Sport,
Aeronautics,NaN
Alpine Skiing,72.068110
Alpinism,NaN
Archery,70.011135
Art Competitions,75.290909
...,...
Tug-Of-War,95.615385
Volleyball,78.900214
Water Polo,84.566446


In [56]:
df.groupby('Sport')['Weight'].agg(['mean'])

,mean
Sport,
Aeronautics,NaN
Alpine Skiing,72.068110
Alpinism,NaN
Archery,70.011135
Art Competitions,75.290909
...,...
Tug-Of-War,95.615385
Volleyball,78.900214
Water Polo,84.566446
